In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
torch.cuda.current_device()

0

In [5]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [6]:
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', './config/cv11Lu_asr_lstm4atthead_allvocab-Finetune02.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'ckpt/')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', './ckpt/cv11Lu_asr_lstm4atthead_allvocab_sd0/best_att-VAG01.pth')
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',8)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', False)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 8)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', True)
    force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from test_asr import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    solver = Solver(config, paras, mode)
    solver.load_data()
    solver.set_model()
    solver.exec()


In [ ]:
main()


Using train mode

[INFO] Exp. name : cv11Lu_asr_lstm4atthead_allvocab-Finetune02_sd0                                         
[INFO] Loading data... large dataset may took a while.                                                     
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Load text encoder : CharacterTextEncoder
Interface for creating all kinds of dataset
import Mozillacv11Dataset as Dataset
[Mozillacv11Dataset] path: /mnt/usb/jason3/cv-corpus-11.0-2022-09-21/zh-TW/wavs, split: ['VAG01CTT']
Mozillacv11Dataset VAG01CTT found wav data: 20
text len: 20
remove None, then wav data: 20, text len: 20
[Mozillacv11Dataset] path: /mnt/usb/jason3/cv-corpus-11.0-2022-09-21/zh-TW/wavs, split: ['VAG01read']
Mozillacv11Dataset VAG01read found wav data: 7
text len: 7
remove None, then wav data: 7, text len: 7
[INFO] Data spec. | Corpus = mozilla_cv11 (from /mnt/usb/jason3/cv-corpus-11.0-2022-09-21/zh-TW/wavs)      
[INFO]  

self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.819 sec/step (rd 16.6% | fw 63.6% | bw 19.8%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.932 sec/step (rd 15.1% | fw 65.2% | bw 19.7%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.959 sec/step (rd 15.5% | fw 65.5% | bw 18.9%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 

self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.813 sec/step (rd 16.7% | fw 63.3% | bw 20.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 1.008 sec/step (rd 14.3% | fw 66.2% | bw 19.6%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.957 sec/step (rd 14.5% | fw 66.6% | bw 18.9%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 

self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.894 sec/step (rd 16.7% | fw 63.5% | bw 19.8%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.935 sec/step (rd 15.3% | fw 65.7% | bw 19.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.861 sec/step (rd 16.4% | fw 63.2% | bw 20.4%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.932 sec/step (rd 15.0% | fw 65.7% | bw 19.3%)

self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.777 sec/step (rd 17.7% | fw 62.2% | bw 20.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.966 sec/step (rd 15.6% | fw 64.5% | bw 19.8%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.928 sec/step (rd 15.6% | fw 65.8% | bw 18.6%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.960 sec/step (r

...

self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.718 sec/step (rd 19.3% | fw 60.8% | bw 20.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.849 sec/step (rd 16.3% | fw 63.8% | bw 19.9%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 1.053 sec/step (rd 14.7% | fw 67.0% | bw 18.3%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.900 sec/step (rd 15.8% | fw 65

self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 1.026 sec/step (rd 14.9% | fw 66.1% | bw 19.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.762 sec/step (rd 18.2% | fw 62.5% | bw 19.4%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 1.026 sec/step (rd 14.2% | fw 65.8% | bw 20.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.947 sec/step (rd 16.1% | fw 65

self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.914 sec/step (rd 16.2% | fw 64.8% | bw 19.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.761 sec/step (rd 18.0% | fw 62.0% | bw 20.0%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.971 sec/step (rd 14.5% | fw 65.8% | bw 19.7%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 

self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 1.027 sec/step (rd 14.2% | fw 67.6% | bw 18.2%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 1.111 sec/step (rd 13.3% | fw 67.2% | bw 19.5%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.800 sec/step (rd 18.6% | fw 62.9% | bw 18.5%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.854 sec/step (rd 16.4% | fw 63

self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.772 sec/step (rd 18.0% | fw 62.8% | bw 19.3%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.854 sec/step (rd 16.4% | fw 63.5% | bw 20.1%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 1.011 sec/step (rd 14.3% | fw 66.9% | bw 18.9%)
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7
self.tr_set: 7 stat | Loss - 0.00 | Grad. Norm - 0.00 | 0.709 sec/step (rd 20.7% | fw 59

In [ ]:
print('asd')

In [11]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv


============  Result of /home/jupyter-jason3/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv ============
 -----------------------------------------------------------------------
| Statics		|  Truth	|  Prediction	| Abs. Diff.	|
 -----------------------------------------------------------------------
| Avg. # of chars	|  8.01	|  8.10	|  0.16		|
| Avg. # of words	|  1.00	|  1.00	|  0.00		|
 -----------------------------------------------------------------------
 ---------------------------------------------------------------
| Error Rate (%)| Mean		| Std.		| Min./Max.	|
 ---------------------------------------------------------------
| Character	| 7.9479	| 21.68		| 0.00/240.00	|
| Word		| 18.1725	| 38.56		| 0.00/100.00	|
 ---------------------------------------------------------------
Note : If the text unit is phoneme, WER = PER and CER is meaningless.

